In [1]:
from transformers import BloomTokenizerFast, BloomForQuestionAnswering, BloomForCausalLM, TrainingArguments, Trainer
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate


/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [10]:

# Save it into pinecone
API_KEY = os.environ.get("PINECONE_API_KEY")
YOUR_ENV = os.environ.get("PINECONE_ENVIRONMENT", "us-west4-gcp-free")
index_name = "test-langchain"
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

In [4]:
pinecone.init(
    api_key=API_KEY,
    environment=YOUR_ENV
)

if len(pinecone.list_indexes()) == 0:
    pinecone.create_index(name=index_name, metric="cosine", shards=1, dimension=len(res[0]))

pinecone.describe_index(pinecone.list_indexes()[0])

IndexDescription(name='test-langchain', metric='cosine', replicas=1, dimension=384.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [7]:
text_field = "text"

index = pinecone.Index(index_name)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [8]:
vectorstore = Pinecone(index, embeddings.embed_query, text_field)

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [52]:
vectorstore.similarity_search(
    "NVIDIA",  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='The GX4000 was a console made by Amstrad in 1990. At first, it was successful. But it did not continue its early success. The GX4000 games could be used with other Amstrad computers. Only 40 games were made for it. Soon it disappeared into the mist of history. These days it is very rare to see one that is still in working condition.\n\nVideo game consoles\n\nde:Amstrad GX4000', metadata={'chunk': 0.0, 'source': 'https://simple.wikipedia.org/wiki/GX4000', 'title': 'GX4000', 'wiki-id': '17530'}),
 Document(page_content='Video games are electronic games played on a video screen (normally a television, a built-in screen when played on a handheld machine, or a computer).\n\nThere are many types, or genres, of these games: role-playing games; shooters, first-person shooters, side-scrollers, and platformers are just a few. \n\nVideo games usually come on CDs, DVDs or digital download. Many games used to come on cartridges. A specialized device used to play a video game

In [28]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [48]:
query = "What is NVIDIa?"


In [49]:
response = qa_with_sources(query)

In [50]:
response['answer']

'NVIDIA is a company that specializes in graphics processing units (GPUs) and is known for its development of the OpenGL programming interface. OpenGL is an open-source API used for 3D graphics programming and is owned by the Khronos Group. It is often compared to Direct3D, another API for 3D graphics on Windows. The GP2X is a handheld gaming console that uses the Linux operating system and can play music, videos, games, and pictures. It can also emulate various video game systems. Kinect is a major accessory developed by Microsoft for the Xbox 360, allowing users to control games without a controller. It was initially called Project Natal and won many awards. Fangames are computer games created by fans based on popular video games, often made using programming languages like C++ or game development tools like Unity or Unreal Engine.\n\n'

In [51]:
response['sources']

'- https://simple.wikipedia.org/wiki/OpenGL\n- https://simple.wikipedia.org/wiki/GP2X\n- https://simple.wikipedia.org/wiki/Xbox%20360\n- https://simple.wikipedia.org/wiki/Fangame'